# Preprocess NSF OCE award data
Created by Ivan Lima on Wed Jan 17 2018 15:47:11 -0500

Last modified on Thu Aug 16 2018 16:49:16 -0400

**In this notebook we remove duplicates records and records with missing abstracts, remove html tags and empty strings from text fields, convert currency strings to numbers, adjust award values for inflation, and compute additional relevant fields for topic modeling of NSF OCE awards**

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 50

## Read award records

In [2]:
awards = pd.read_csv('data/nsf_oce_awards_1985-2017_clean.csv', parse_dates=[4,5,11], encoding="ISO-8859-1")
print('Data columns:\n{}'.format(awards.columns))

Data columns:
Index(['AwardNumber', 'Title', 'NSFOrganization', 'Program(s)', 'StartDate',
       'LastAmendmentDate', 'PrincipalInvestigator', 'State', 'Organization',
       'AwardInstrument', 'ProgramManager', 'EndDate', 'AwardedAmountToDate',
       'Co-PIName(s)', 'PIEmailAddress', 'OrganizationStreet',
       'OrganizationCity', 'OrganizationState', 'OrganizationZip',
       'OrganizationPhone', 'NSFDirectorate', 'ProgramElementCode(s)',
       'ProgramReferenceCode(s)', 'ARRAAmount', 'Abstract'],
      dtype='object')


### Rename some variables

In [3]:
awards = awards.rename(index=str, columns={'Program(s)':'Programs',
                                           'PrincipalInvestigator':'PI',
                                           'AwardInstrument':'Instrument',
                                           'AwardedAmountToDate':'Amount',
                                           'ARRAAmount':'ARRA',
                                           'ProgramElementCode(s)':'ElementCodes',
                                           'ProgramReferenceCode(s)':'ReferenceCodes'})

### Compute number of Co-PIs for each award

In [4]:
awards['num_copi'] = [len(cp.split(',')) if cp not in [np.nan] else np.nan for cp in awards['Co-PIName(s)']]

## Clean & preprocess data

Bayesian probabilistic models are quite sensitive to the input data. Relatively small changes in the collection of documents or on how the *bag-of-words* is built can change some of the topics extracted.

**Note:** Collaborative Research awards are those in which investigators from two or more organizations collaborate on one research project. These awards share the same abstract but the different organizations receive separate awards. Here we consider one abstract per project and sum the amounts awarded to each organization.

In [5]:
awards['Abstract'][awards.AwardNumber==9421772] = np.nan  # renewal award
ntot = len(awards)

# remove duplicate records
awards_clean = awards.drop_duplicates('AwardNumber')
nodup = len(awards_clean)

# remove entries with no abstracts
awards_clean = awards_clean[awards_clean['Abstract'].notnull()]
nomiss = len(awards_clean)

# select variables to keep
awards_clean = awards_clean[['AwardNumber', 'Title', 'Programs', 'StartDate', 'EndDate', 'ProgramManager',
                             'Instrument', 'PI', 'Organization', 'Amount', 'ARRA', 'ElementCodes', 'ReferenceCodes', 
                             'Abstract', 'num_copi']]

# convert currency strings to numbers
awards_clean['Amount'] = [np.float(s.replace(',','').lstrip('$')) for s in awards_clean.Amount.values]
awards_clean['ARRA']   = [np.float(s.replace(',','').lstrip('$')) for s in awards_clean.ARRA.values]

# remove html tags from abstracts
awards_clean['Abstract'] = [s.replace('<br/>',' ') for s in awards_clean.Abstract.values]

# remove common words
for word in ['OCE','ABSTRACT','Abstract','PROJECT','Project']:
    awards_clean['Abstract'] = [s.replace(word,' ') for s in awards_clean.Abstract.values]

# remove award numbers from abstracts
abstracts = awards_clean.Abstract.values
for n in awards_clean.AwardNumber.unique():
    abstracts = [s.replace(str(n),' ') if str(n) in s else s for s in abstracts]
awards_clean['Abstract'] = abstracts

# remove empty spaces from abstracts & titles
awards_clean['Abstract'] = [' '.join(s.split()) for s in awards_clean.Abstract.values]
awards_clean['Title'] = [' '.join(s.split()) for s in awards_clean.Title.values]

# clean empty strings from Programs & convert to upper case
awards_clean['Programs'] = [','.join(list(filter(bool,s.split(', ')))).upper() 
                            for s in awards_clean.Programs.fillna('NAN')]
awards_clean.loc[awards_clean.Programs=='NAN','Programs'] = np.nan

# get number of programs for each award
awards_clean['num_programs'] = [len(s.split(',')) for s in awards_clean.Programs.fillna('NAN')]
awards_clean.loc[awards_clean.Programs.isnull(),'num_programs'] = np.nan

# set award program to first on the list
awards_clean['program'] = [s.split(',')[0] for s in awards_clean.Programs.fillna('NAN')]
awards_clean.loc[awards_clean.Programs.isnull(),'program'] = np.nan

work = awards_clean.drop_duplicates('Abstract')
anum = work.AwardNumber.values

# sum collaborative research awards into one total amount
total = awards_clean.groupby('Abstract')['Amount'].sum()
total = total.sort_index()
awards_final = awards_clean.drop_duplicates('Abstract')
awards_final = awards_final.sort_values('Abstract')
awards_final['total_amount'] = total.values
awards_final = awards_final.set_index('AwardNumber').loc[anum,:].reset_index()
nclean = len(awards_final)

print('Number of duplicate records: {}'.format(ntot - nodup))
print('Number of records with missing abstracts: {}'.format(nodup - nomiss))
print('Number of Collaborative Research awards with repeating abstracts: {}'.format(nomiss - nclean))
print('Total removed (missing + duplicate + Collaborative Research): {}\n'.format(ntot - nclean))
print('Size of original dataset: {} records'.format(ntot))
print('Size of final preprocessed dataset: {} records'.format(nclean))

Number of duplicate records: 4755
Number of records with missing abstracts: 1212
Number of Collaborative Research awards with repeating abstracts: 2660
Total removed (missing + duplicate + Collaborative Research): 8627

Size of original dataset: 19865 records
Size of final preprocessed dataset: 11238 records


## Adjust amount awarded for inflation

In [6]:
cpi = pd.read_excel('data/CPI.xlsx', skiprows=range(11), index_col=0) # consumer price index
rel_cpi = cpi.Annual/cpi.loc[2017,'Annual']                           # use 2017 as reference year
rel_cpi_map = {k:v for k, v in zip(rel_cpi.index, rel_cpi.values)}
rel_cpi_map[2018] = 1.0                                               # add 2018
awards_final['amount_adjusted'] = awards_final.total_amount / awards_final.StartDate.map(lambda x: rel_cpi_map[x.year])
awards_final['arra_adjusted']   = awards_final.ARRA / awards_final.StartDate.map(lambda x: rel_cpi_map[x.year])

# remove suspiciously low values (mainly zeros) that might bias the annual means
awards_final.loc[awards_final.amount_adjusted<1.e+3,['Amount','amount_adjusted']] = np.nan

## Save preprocessed data to HDF5 file

In [7]:
awards_final = awards_final.reset_index(drop=True)
store = pd.HDFStore('data/nsf_oce_awards_1985-2017.h5',complevel=9)
store['awards'] = awards_final
store.close()